# Laboratory 03 - Introduction to Digital Data Acquisition, FFT,  and Spectrum Analysis 2

## MAE 3120, Spring 2020

## Grading Rubric

Procedures, Results, Plots, Tables - 60%

Discussion Questions - 25%

Neatness - 15%

## Introduction and Background

Prior to the 1980s, the oscilloscope and strip-chart recorder represented the most common methods for measurement of time-varying signals. With time PC-based digital data acquisition became standard in most laboratories. By combining high-speed data acquisition cards with graphical software, it is now possible to design complex data acquisition systems with real-time data analysis and plotting features, with minimal programming. The data acquisition hardware converts analog inputs into the digital domain at the specified sampling rate, and the software manipulates and displays the desired output. 

In this lab we use Python and the NI-DAQmx API for digital data acquisition. Using the ***DAQ*** Jupyter Notebook developed for this class, instructions are issued to the data acquisition hardware, either inside the PC or external to the PC (the hardware we use in our lab is connected through the USB port). The ***DAQ*** can be configured to record data to files, change sampling parameters, and display a live output of your sampled signal. 

The goal of this tutorial is to provide you with your first experience using the ***DAQ*** notebook to perform data acquisition. You will use the ***DAQ*** to take samples and plot voltage data and to illustrate some limitations of digital data acquisition systems. 

To help verify that you have configured the ***DAQ*** properly before performing trials, you will learn how to use ***NI MAX*** (a software provided by National Instruments). 

Ultimately, you will experiment with digital data acquisition and some of its shortcomings. For your report you are expected to save all the data you will acquire in the lab to files and plot them in Python. 

___

In spectral analysis the goal is to determine the frequency content of a signal. Aliasing can be a serious problem with digital data acquisition if the experimenter is not careful. Significant measurement errors called ***aliasing*** errors are possible if the waveform is not sampled at high enough frequency. To avoid aliasing, the ***sampling rate*** must be at least twice the maximum frequency of the measured signal. This restriction is called the ***Nyquist criterion***. Signal aliasing occurs when waveforms are sampled at frequencies below the Nyquist frequency. Aliased signals appear to have frequencies (and possibly even waveform *shapes*) that differ from those of the actual signal. For adequate resolution of the waveform shape, data should be sampled at a much higher frequency – typically at least five times the Nyquist frequency, if possible.

Digital PC-based data acquisition will not totally replace oscilloscopes, at least not in the near future. The reason is sampling frequency. The maximum sampling frequency of modern PC A/D systems is typically less than a MHz (megahertz). By comparison, a good digital oscilloscope may sample as high as several GHz (gigahertz)!

The fast Fourier transform (FFT) is a computationally efficient form of the more general discrete Fourier transform (DFT), which is itself a discretized version of the even more general Fourier transform (FT). Like Fourier series analysis, FFT analysis enables us to calculate the frequency content of a signal. Fourier series analysis is useful for continuous, periodic, analog signals of known fundamental frequency. FFT analysis, on the other hand, is useful for discretely sampled (digital) data, and can be applied even if the signal is not periodic. With FFT analysis, the fundamental frequency of a periodic signal does not have to be known a priori. NumPy has built-in FFT features, which are utilized in this lab. 

For $N$ sampled data points at sampling frequency $f_s$, the most useful output of an FFT calculation is the frequency spectrum or amplitude spectrum, which is a plot of modified FFT amplitude versus frequency. The frequency spectrum shows the relative importance or contribution of discrete frequencies, which range from zero to $f_s\,/\,2$. (The factor of two is a direct result of the Nyquist criterion.) The number of discrete frequencies on the frequency spectrum plot is $N\,/\,2 + 1$. This is half of the number of discretely sampled data points in the original signal, plus one extra since we typically plot both extreme values – from zero Hz (DC component) to the folding frequency $f_\textit{folding}$. 

Here are some useful definitions for FFTs:

- $N$ is the ***total number of discrete data points*** taken. $N$ is an input parameter, chosen by the user.<br><p></p>

- $f_s$ is the ***sampling frequency***, in Hz. $f_s$ is an input parameter, chosen by the user. *All other properties of the FFT, including sampling time, maximum frequency, frequency resolution, etc., are determined solely from these two inputs, $N$ and $f_s$.*<br><p></p>

- $T$ is the ***total sampling time***, and is calculated as $T = N\,/\,f_s$. To increase the sampling time, we must either *increase* the number of data points, or *decrease* the sampling frequency (or both).<br><p></p>

- $f_\textit{folding}$ is the ***folding frequency***, also called $f_\textit{max}$, the ***maximum frequency***. $f_\textit{folding} = f_s\,/\,2$. $f_\textit{folding}$ is the maximum frequency plotted on the frequency spectrum plot, since $f_\textit{folding}$ is the maximum frequency at which reliable information about the signal can be calculated, due to the Nyquist criterion. The only way to increase $f_\textit{folding}$ is to increase the sampling frequency.<br><p></p>

- $\Delta f$ is the ***frequency resolution*** or ***frequency increment*** of the frequency spectrum. $\Delta f = 1\,/\,T = f_s\,/\,N$. On the frequency spectrum plot, amplitudes of the FFT are plotted at $N\,/\,2 + 1$ discrete frequencies, each separated by $\Delta f$. In other words, the discrete values of $f$ are $0$, $\Delta f$, $2 \Delta f$, $3 \Delta f$, ... , $[(N\,/\,2\,– 1)] \Delta f$. (The amplitude at exactly $f_\textit{folding}$, i.e., at $(N\,/\,2) \Delta f$, is also plotted; this results in a total of $(N\,/\,2) + 1$ discrete frequencies, counting both $f = 0$ and $f = f_\textit{folding}$). The *only* way to increase the frequency resolution is to increase sampling time.<br><p></p>

Here is a summary of some useful techniques and rules to remember when calculating FFTs:

- To get better frequency resolution for a fixed sampling frequency, increase the number of data points.<br><p></p>

- To get better frequency resolution for a fixed number of data points, decrease the sampling frequency. (But be careful here not to let $f_s$ fall below the Nyquist criterion limit).<br><p></p>

- To get frequency component information at higher frequencies, increase the sampling frequency.<br><p></p>

- To reduce ***leakage*** in the frequency spectrum, do one or more of the following:<br><p></p>

  - Increase the number of sampled data points $N$ (at the cost of more computer time).<br><p></p>
  
  - Decrease the sampling frequency $f_s$ (but do not sample at such a low frequency that the Nyquist criterion is violated).<br><p></p>
  
  - Multiply the time signal by a ***windowing*** function prior to taking the FFT (at the cost of throwing away a significant portion of the signal, in particular data points near the start and finish of the time trace).

## Objectives

- Practice data acquisition with digital data acquisition systems.<br><p></p>

- Construct simple first-order anti-aliasing (low-pass passive) filters.<br><p></p>

- Learn a simple way to sum two voltage signals.<br><p></p>

- Examine the usefulness of an anti-aliasing filter.

## Equipment

- Computer<br><p></p>

- Software: NI MAX, Jupyter<br><p></p>

- Hardware: National Instrument CompactDAQ cDAQ-9174, NI-9201 C Series Voltage Input Module <br><p></p>

- Function/waveform generator, along with appropriate cables<br><p></p>

- Oscilloscope<br><p></p>

- Resistors (value to be determined by you)<br><p></p>

- 2 capacitors of nominal value .1 μF (.1 × 10<sup>-6</sup> F)<br><p></p>

- Breadboard (plugged in for the ground connection, but turned off – power not needed for this lab)<br><p></p>

- Various BNC and banana cords and breadboard jumper wires as needed

## Procedure

### Part I - Clipping

1. Remember to be organized when acquiring data. Generate a table that includes all the runs and associated parameters you will acquire in this lab and report this matrix of experiments in your lab report.<br><p></p>

- Connect the output of the function generator to the oscilloscope and also to the first channel of the *NI 9201* module using BNC cables and a BNC T-adapter. <br><p></p>

- Turn on the function generator and oscilloscope. <br><p></p>

- Set the function generator to produce a 100 Hz sine wave, 0V DC Offset, and 5V peak-to-peak amplitude. Check the signal using the oscilloscope. <br><p></p>

- Set the sampling rate to 10,000 Hz and the number of samples to 1000 in order to record 100 ms of data (10 full waveforms). Acquire one trial of data and save it to a file using the `acquire` function. <br><p></p>

- Using a 'live' output, adjust the DC offset and/or amplitude of the signal produced by the waveform generator to observe how the digital signal is clipped.<br><p></p>

- When you have a display that clearly illustrates clipping, stop the live output and acquire 100 ms of data. Ensure the data is saved to a file for use in your lab report. You should also report the DC offset, amplitude, and any other relevant waveform generator settings.

### Part II - Aliasing

Here you will develop an anti-aliasing filter and verify its behavior. You will use the same circuit for *Part III*, so do not modify your circuit after this section.<br><p></p>

1. Using the same configuration than previously ($f_s = 51.2\text{ Hz}$), now input a waveform of $f = 40\text{ Hz}$ and demonstrate aliasing. Estimate the apparent frequency analytically and compare it to the measured data. Take screenshots for your report.<br><p></p>

- You will now construct a first-order low-pass filter using a combination of resistors and capacitors. A circuit diagram for a simple first-order low-pass filter is provided below, the cutoff frequency is determined by: $f_\textit{cutoff} = \large \frac{1}{2\pi RC}$. Set the cutoff frequency at the sampling frequency record this signal without aliasing. Select the value of the resistor accordingly. Note: In all these circuits, it is critical that the ground bus be connected to a physical ground, such as the black ground post of the powered breadboard with the breadboard plugged in (but turned off) to secure the ground. <br><p></p>

- Test your filter by connecting the function generator to the filter input and the filter output to the DAQ board. Start with a sine wave input of about 5 Hz and amplitude 1V. Increase the frequency of the sine wave, doubling it every time, until you start noticing that the output amplitude decreases – the low-pass filter is doing its job. Keep increasing the frequency until the filter output amplitude is 90% attenuated – in other words, G = 0.1; the output amplitude is about 10% of the input amplitude, or 0.1V. At what frequency does this occur?

<img src="img/FFT%20Circuit.png" width=140>

### Part III - Signal Reconstruction

A signal contaminated with a high-frequency noise will be simulated and its proper acquisition with an anti-aliasing filter studied in this section. This requires using advanced function in the waveform generator to generate the sum of two signals. The carrier wave is a sine wave, 10 Hz, 5V. The noise is a sine wave at 3.1 kHz with amplitude 1V. <br><p></p>

1. Program the sum of the two sines in the waveform generator. In order to sum high frequency noise to carrier signal, go to modulate button on waveform function generator. Turn on modulate and choose sum under type, internal under source. Choose sine as shape of the noise and give the sum amplitude and frequency as it mentioned above.<br><p></p>

- Monitor that you have the proper signal on the oscilloscope. <br><p></p>

- Send the signal directly to the DAQ system (i.e. without going through the anti-aliasing filter that you have created).<br><p></p> 

- Sample at 500 Hz with 1,024 data points per scan. You should observe the low frequency signal nicely, but the high frequency signal should yield some aliasing.<br><p></p>

- Save the time trace and recreate the frequency spectrum for your lab report. Estimate the frequency of the two signals from the frequency spectrum plot. Calculate the frequency resolution of your DAQ system for this sampling frequency and comment on the resolution of your signal.<br><p></p>

- Redo *Steps 4 & 5* with a sampling frequency of:<br><p></p>
  - 1 kHz <br><p></p>
  
  - 5 kHz<br><p></p>
  
  - 10 kHz<br><p></p>
  
- Run the summed signal through the low-pass filter created previously. The low-pass filter is now acting as an anti-aliasing filter. Predict the amplitude of the noise and compare to the quantization error of your DAQ board.<br><p></p>

- Reacquire signals for each sampling frequency, i.e. $f_s$ = 0.5, 1, 5, and 10 kHz. Record these traces into your lab report as well. Take particular note of the high frequency noise. Has it been reduced? Has it been totally eliminated? Record your observations. 

# Discussion Questions

1. Which statistical tool/graph could you use to identify if some clipping took place in your data? What would you expect to see? <br><p></p>

2. For *Part III*, did your anti-aliasing filter effectively remove the high frequency “noise”, while at the same time preserving the desired low frequency signal? Why or why not? Explain.<br><p></p>

3. In *Part III*, which frequency was the optimum to record your signal? Hint: think about the frequency resolution.

# Appendices 

## Appendix A - NI cDAQ-9174

<img src="img/cDAQ-9174.png" width=240 align="left"><br><br><br><br><br><br><br><br>

[Online Manual](https://www.ni.com/documentation/en/compactdaq-chassis/latest/cdaq-9174/overview/)

[User Manual](https://www.ni.com/pdf/manuals/372838e.pdf)

[Specification Sheet](https://www.ni.com/pdf/manuals/374045a.pdf)

## Appendix B - NI 9201

<img src="img/NI-9201.png" width=150 align="left"><br><br><br><br><br><br><br><br>

[HTML Manual](https://www.ni.com/documentation/en/c-series-voltage-input-module/latest/9201/overview/)

[Datasheet](https://www.ni.com/pdf/manuals/373783a_02.pdf)

**Signal Level**: ± 10V

**Channels**: 8 Single-Ended

**Max Sample Rate (Single Channel)**: 800 kS/s

**Max Sample Rate (Scanning)**: 500 kS/s

**Simultaneous** No

**ADC Resolution**: 12-Bit

**Type of ADC**: Successive approximation register (SAR)

<img src="img/NI-9201%20Circuit.png" width=480 align="left"><br><br><br><br><br><br><br><br><br>

<img src="img/NI-9201%20Sample%20Rate.png" width=480 align="left"><br><br><br><br><br><br><br><br><br><br><br><br>

<img src="img/NI-9201%20Accuracy.png" width=480 align="left"><br><br><br><br><br><br>

<img src="img/NI-9201%20Stability.png" width=480 align="left">